# Comparing canonical functional networks vs. complex structural eigenmodes.

In [1]:
from ipywidgets import interactive, widgets, fixed
from surfer import Brain as surface
from sklearn.preprocessing import minmax_scale

import os
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# spectrome imports
from spectrome.brain import Brain
from spectrome.utils import functions, path

In [2]:
%gui qt

# set up Pysurfer variables
subject_id = "fsaverage"
hemi = ["lh","rh"]
surf = "white"

"""
Read in the automatic parcellation of sulci and gyri.
"""

hemi_side = "lh"
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi_side + ".aparc.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

In [3]:
# function for viewing canonical networks:
def get_fc_values(fc_df, labels, fc_name):
    # get our data ready in both hemispheres
    fc_network = fc_df.loc[fc_name].values
    lh_cort = minmax_scale(fc_network[0:34])
    rh_cort = minmax_scale(fc_network[34:68])

    # for pysurfer requirements
    lh_pad = np.insert(lh_cort, [0, 3], [0, 0])
    rh_pad = np.insert(rh_cort, [0, 3], [0, 0])

    lh_fc = lh_pad[labels]
    rh_fc = rh_pad[labels]

    fc_brain = surface(
        subject_id,
        "both",
        surf,
        background="white",
        alpha=0.3,
        title="Canonical Networks",
    )
    fc_brain.add_data(lh_fc, hemi="lh", thresh=0.15, colormap=plt.cm.autumn_r, remove_existing=True)
    fc_brain.add_data(rh_fc, hemi="rh", thresh=0.15, colormap=plt.cm.autumn_r, remove_existing=True)
    fc_brain.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent=False)
    return lh_fc, rh_fc

In [4]:
fc_names = [
    "Visual",
    "Limbic",
    "Default",
    "Somatomotor",
    "Frontoparietal",
    "Ventral_Attention",
    "Dorsal_Attention",
]

color_fmin, color_fmid, color_fmax = 0.1, 0.5, 0.9

# Load Pablo's canonical networks in DK atlas:
fc_dk = np.load("../data/com_dk.npy", allow_pickle=True).item()
fc_dk_normalized = pd.read_csv("../data/DK_dictionary_normalized.csv").set_index(
    "Unnamed: 0"
)

In [5]:
interactive(
    get_fc_values,
    fc_df=fixed(fc_dk_normalized),
    labels=fixed(labels),
    fc_name=widgets.RadioButtons(
        options=fc_names, value="Limbic", description="Select canonical network"
    ),
)

colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [1.00e-01, 5.00e-01, 9.00e-01] (opaque)


In [6]:
## This for-loop is for generating and saving figures for the paper don't run unless you really want to.
## Whatever is being generated here you can get with the interactive widget in the previous cell

for names in fc_names:
    lh, rh = get_fc_values(fc_dk_normalized, labels = labels, fc_name = names)
    # Generate FOV figures for 1 hemisphere first
    sb = surface(subject_id, 'lh', surf, background = "white", alpha = 1, title = "Canonical Network")
    sb.add_data(lh, hemi = 'lh', thresh = 0.15, colormap = plt.cm.autumn_r, remove_existing = True)
    sb.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent = False)
    sb.show_view('lat')
    sb.save_image('%s_lat.svg' % names)
    sb.show_view('med')
    sb.save_image('%s_med.svg' % names)
    # Generate FOV for both hemisphere dorsal view
    sb = surface(subject_id, "both", surf, background = "white", alpha = 1, title = "Canonical Network")
    sb.add_data(rh, hemi = 'rh', thresh = 0.15, colormap = plt.cm.autumn_r, remove_existing = True)
    sb.add_data(lh, hemi = 'lh', thresh = 0.15, colormap = plt.cm.autumn_r, remove_existing = True)
    sb.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent = False)
    ## save figures?
    sb.show_view('dor')
    sb.save_image('%s_dor.svg' % names)

colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [1.00e-01, 5.00e-01, 9.00e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [1.00e-01, 5.00e-01, 9.00e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [1.00e-01, 5.00e-01, 9.00e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [1.00e-01, 5.00e-01, 9.00e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [1.00e-01, 5.00e-01, 9.00e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [1.00e-01, 5.00e-01, 9.00e-01] (opaque)
colormap sequential: [0.

### Now we visualize the best matching complex structural eigenmodes for each network

In [17]:
## Load the optimized parameters first
data_path = "../data"
h5_path = os.path.join(data_path, "default.h5")
bh_default = path.read_hdf5(h5_path)
print('Default network parameters:' + str(np.round(bh_default['x'],2)) + ' at R=' + str(-np.round(bh_default['fun'],2)))

h5_path = os.path.join(data_path, "dorsal.h5")
bh_dorsal = path.read_hdf5(h5_path)
print('Doral Attention network parameters:' + str(np.round(bh_dorsal['x'],2)) + ' at R=' + str(-np.round(bh_dorsal['fun'],2)))

h5_path = os.path.join(data_path, "fronto.h5")
bh_front = path.read_hdf5(h5_path)
print('Frontoparietal network parameters:' + str(np.round(bh_front['x'],2)) + ' at R=' + str(-np.round(bh_front['fun'],2)))

h5_path = os.path.join(data_path, "limbic.h5")
bh_limbic = path.read_hdf5(h5_path)
print('Limbic network parameters:' + str(np.round(bh_limbic['x'],2)) + ' at R=' + str(-np.round(bh_limbic['fun'],2)))

h5_path = os.path.join(data_path, "motor.h5")
bh_motor = path.read_hdf5(h5_path)
print('Somatomotor network parameters:' + str(np.round(bh_motor['x'],2)) + ' at R=' + str(-np.round(bh_motor['fun'],2)))

h5_path = os.path.join(data_path, "ventral.h5")
bh_ventral = path.read_hdf5(h5_path)
print('Ventral Attention network parameters:' + str(np.round(bh_ventral['x'],2)) + ' at R=' + str(-np.round(bh_ventral['fun'],2)))

h5_path = os.path.join(data_path, "visual.h5")
bh_visual = path.read_hdf5(h5_path)
print('Visual network parameters:' + str(np.round(bh_visual['x'],2)) + ' at R=' + str(-np.round(bh_visual['fun'],2)))

Default network parameters:[ 2.97 30.41] at R=0.61
Doral Attention network parameters:[ 2.6  78.54] at R=0.61
Frontoparietal network parameters:[ 2.93 42.27] at R=0.66
Limbic network parameters:[ 3.93 74.85] at R=0.56
Somatomotor network parameters:[  1. 100.] at R=0.6
Ventral Attention network parameters:[ 3.32 60.76] at R=0.58
Visual network parameters:[  3.88 249.44] at R=0.62


Plot networks:

In [39]:
from scipy.stats import spearmanr


def pysurfer_prep(pysurf_in, labels, atlas="DK"):
    scaled_in = minmax_scale(pysurf_in)
    if atlas == "DK":
        padded = np.insert(scaled_in, [0, 3], [0, 0])
    else:
        padded = scaled_in

    pysurf_out = padded[labels]
    return pysurf_out


def eigmode2plot(labels, alpha_optimized, k_optimized, fc_name, lap_type="complex"):
    hcp_dir = "../data"
    thr_colors = 0.35
    # Compute eigenmode dice with Brain:
    brain = Brain.Brain()
    brain.add_connectome(hcp_dir)
    brain.reorder_connectome(brain.connectome, brain.distance_matrix)
    brain.bi_symmetric_c()
    brain.reduce_extreme_dir()
    if lap_type == "complex":
        brain.decompose_complex_laplacian(
            alpha=alpha_optimized, k=k_optimized, num_ev=86
        )
    elif lap_type == "real":
        brain.add_regular_laplacian_eigenmodes(
            alpha=alpha_optimized, k=k_optimized, num_ev=86, vis=False
        )

    # Compute the spearman correlation again for both single eigenmode:
    canon_network = np.nan_to_num(fc_dk_normalized.loc[fc_name].values)
    corrs = np.squeeze(np.zeros([brain.norm_eigenmodes.shape[1], 1]))
    for e in np.arange(0, len(corrs)):
        spearman_corr = spearmanr(
            np.squeeze(canon_network), brain.norm_eigenmodes[:, e]
        )[0]
        corrs[e] = spearman_corr

    # Sort eigenmode by performance:
    ntw_opt_corr = np.round(corrs, 3)
    ordered_corr = np.argsort(-ntw_opt_corr)

    # For single best eigenmode:
    K = 1
    canon_network = np.nan_to_num(fc_dk_normalized.loc[fc_name].values).reshape(-1, 1)
    corr_eigs = brain.norm_eigenmodes[:, ordered_corr[0:K]]

    # prepare eigmodes for pysurfer:
    lh_best = pysurfer_prep(corr_eigs[0:34], labels)
    rh_best = pysurfer_prep(corr_eigs[34:68], labels)

    # For top 10 combined:
    K = 10
    corr_eigs = brain.norm_eigenmodes[:, ordered_corr[0:K]]
    coef, r, _, _ = np.linalg.lstsq(corr_eigs, canon_network, rcond=None)
    comb_eig = np.squeeze(np.matmul(corr_eigs, np.asarray(coef)))

    # pysurfer:
    lh_combined = pysurfer_prep(comb_eig[0:34], labels)
    rh_combined = pysurfer_prep(comb_eig[34:68], labels)

    # visualize:
    # best eigenmode first:
    best_min = 0.20+lh_best.min()
    best_max = 0.95*lh_best.max()
    best_mid = 0.70*lh_best.max()
    sb = surface(subject_id, "lh", surf, background="white", alpha=1)
    sb.add_data(lh_best, hemi="lh", thresh=thr_colors, colormap=plt.cm.autumn_r, remove_existing=True)
    sb.scale_data_colormap(best_min, best_mid, best_max, transparent=False)

    ## show lateral and medial views of left hemisphere and save figures
    sb.show_view("lat")
    sb.save_image("%s_ScaledBest_Lat.svg" % fc_name)
    sb.show_view("med")
    sb.save_image("%s_ScaledBest_Med.svg" % fc_name)

    ## dorsal view with both hemispheres:
    sb = surface(subject_id, "both", surf, background="white", alpha=1)
    sb.add_data(rh_best, hemi="rh", thresh=thr_colors, colormap=plt.cm.autumn_r, remove_existing=True)
    sb.add_data(lh_best, hemi="lh", thresh=thr_colors, colormap=plt.cm.autumn_r, remove_existing=True)
    sb.scale_data_colormap(best_min, best_mid, best_max, transparent=False)
    ## save figures?
    sb.show_view("dor")
    sb.save_image("%s_ScaledBest_Dor.svg" % fc_name)

    # combination:
    # best eigenmode first:
    combine_min, combine_max, combine_mid = 0.20+lh_combined.min(), 0.95*lh_combined.max(), 0.75*lh_combined.max()
    sb = surface(subject_id, "lh", surf, background="white", alpha=1)
    sb.add_data(lh_combined, hemi="lh", thresh=thr_colors, colormap=plt.cm.autumn_r, remove_existing=True)
    sb.scale_data_colormap(combine_min, combine_mid, combine_max, transparent=False)

    ## show lateral and medial views of left hemisphere and save figures
    sb.show_view("lat")
    sb.save_image("%s_ScaledCombined_Lat.svg" % fc_name)
    sb.show_view("med")
    sb.save_image("%s_ScaledCombined_Med.svg" % fc_name)

    ## dorsal view with both hemispheres:
    sb = surface(subject_id, "both", surf, background="white", alpha=1)
    sb.add_data(rh_combined, hemi="rh", thresh=thr_colors, colormap=plt.cm.autumn_r, remove_existing=True)
    sb.add_data(lh_combined, hemi="lh", thresh=thr_colors, colormap=plt.cm.autumn_r, remove_existing=True)
    sb.scale_data_colormap(combine_min, combine_mid, combine_max, transparent=False)
    ## save figures?
    sb.show_view("dor")
    sb.save_image("%s_ScaledCombined_Dor.svg" % fc_name)
    return lh_best, rh_best, lh_combined, rh_combined, ordered_corr

limbic network:

In [41]:
lh_best, rh_best, lh_combined, rh_combined, ordered_corr = eigmode2plot(
    labels, bh_default["x"][0], bh_default["x"][1], fc_name="Default"
)
# figures are saved in current directory
print('Best eigenmode is #:' + str(ordered_corr[0]))

colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
Best eigenmode is #:35


Visual:

In [42]:
lh_best, rh_best, lh_combined, rh_combined, ordered_corr = eigmode2plot(
    labels, bh_visual["x"][0], bh_visual["x"][1], fc_name="Visual"
)
# the figures are saved in current directory
print(ordered_corr[0])

colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
19


Frontoparietal:

In [43]:
lh_best, rh_best, lh_combined, rh_combined, ordered_corr = eigmode2plot(
    labels, bh_front["x"][0], bh_front["x"][1], fc_name="Frontoparietal"
)
# the figures are saved in current directory
print(ordered_corr[0])

colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
17


Somatomotor:

In [44]:
lh_best, rh_best, lh_combined, rh_combined, ordered_corr = eigmode2plot(
    labels, bh_motor["x"][0], bh_motor["x"][1], fc_name="Somatomotor"
)
# the figures are saved in current directory
print(ordered_corr[0])

colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
84


Ventral Attention:

In [45]:
lh_best, rh_best, lh_combined, rh_combined, ordered_corr = eigmode2plot(
    labels, bh_ventral["x"][0], bh_ventral["x"][1], fc_name="Ventral_Attention"
)
# the figures are saved in current directory
print(ordered_corr[0])

colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
56


Dorsal Attention:

In [46]:
lh_best, rh_best, lh_combined, rh_combined, ordered_corr = eigmode2plot(
    labels, bh_dorsal["x"][0], bh_dorsal["x"][1], fc_name="Dorsal_Attention"
)
# the figures are saved in current directory
print(ordered_corr[0])

colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.00e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [0.00e+00, 5.00e-01, 1.00e+00] (opaque)
colormap sequential: [2.00e-01, 7.50e-01, 9.50e-01] (opaque)
16
